In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

geojson с муниципальными образованиями от [gis-lab.info](http://gis-lab.info/qa/moscow-atd.html#.D0.A1.D0.BA.D0.B0.D1.87.D0.B0.D1.82.D1.8C_.D0.B4.D0.B0.D0.BD.D0.BD.D1.8B.D0.B5) или [github.com](https://github.com/blackmad/neighborhoods/blob/master/gn-moscow.geojson)

```bash
$ conda install -c conda-forge geopandas
```

Если ошибка во время импорта `fiona`
```bash
$ conda install fiona "libgdal<2.1"

```

Альтернатвный вариант
```bash
$ conda install pandas
$ pip install geopandas

```

In [2]:
shapes = gpd.read_file("./mo.geojson")
shapes = shapes.loc[:,["geometry", "NAME"]]
shapes.head()

,geometry,NAME
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский
1,"POLYGON ((37.42765 55.74821, 37.42849 55.74875...",Филёвский Парк
2,"POLYGON ((36.80357 55.45162, 36.80451 55.46551...",Новофёдоровское
3,"POLYGON ((36.93724 55.24139, 36.93726 55.24161...",Роговское
4,"POLYGON ((37.43956 55.62731, 37.44018 55.63042...","""Мосрентген"""


In [3]:
events = pd.read_table("./data_5m.csv")
events["geometry"] = events.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)
events = gpd.GeoDataFrame(events, geometry="geometry", crs={"init": "CRS84"})  # WGS84
events.head()

,actionid,hours,is_mobile,is_tablet,lat,lon,minuts,rtb_stlm_price,sd_age,sd_gender,sd_income,seconds,geometry
0,0,415593,0,0,55.60096,37.47411,24935582,10738,-1,-1,-1,1496134974,POINT (37.47411 55.60096)
1,0,415594,0,0,55.75064,37.75640,24935654,3301,2,1,2,1496139261,POINT (37.7564 55.75064)
2,0,415604,1,0,55.71684,37.59285,24936246,10670,4,0,0,1496174796,POINT (37.59285 55.71684)
3,0,415598,0,0,55.71103,37.43181,24935882,2801,4,0,2,1496152963,POINT (37.43181 55.71103000000001)
4,0,415593,0,0,55.78307,37.44938,24935625,29905,2,1,1,1496137549,POINT (37.44938 55.78307)


In [6]:
joined_data = gpd.tools.sjoin(shapes, events, how="inner")
joined_data[~joined_data.NAME.isnull()].head()

,geometry,NAME,index_right,actionid,hours,is_mobile,is_tablet,lat,lon,minuts,rtb_stlm_price,sd_age,sd_gender,sd_income,seconds
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,2745164,0,415597,1,0,55.31925,36.90929,24935855,2000,4,0,0,1496151312
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,2845409,0,415597,1,0,55.31925,36.90929,24935860,2000,4,0,0,1496151609
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,3163624,0,415602,1,0,55.42967,36.86978,24936158,4900,4,0,1,1496169517
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,2522094,0,415597,0,0,55.42846,36.86983,24935836,8669,3,1,1,1496150163
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,2805045,0,415597,0,0,55.42846,36.86983,24935839,3746,3,1,1,1496150378


In [7]:
joined_data[joined_data.NAME.isnull()].head()

,geometry,NAME,index_right,actionid,hours,is_mobile,is_tablet,lat,lon,minuts,rtb_stlm_price,sd_age,sd_gender,sd_income,seconds


In [8]:
joined_data[~joined_data.NAME.isnull()].head()

,geometry,NAME,index_right,actionid,hours,is_mobile,is_tablet,lat,lon,minuts,rtb_stlm_price,sd_age,sd_gender,sd_income,seconds
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,2745164,0,415597,1,0,55.31925,36.90929,24935855,2000,4,0,0,1496151312
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,2845409,0,415597,1,0,55.31925,36.90929,24935860,2000,4,0,0,1496151609
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,3163624,0,415602,1,0,55.42967,36.86978,24936158,4900,4,0,1,1496169517
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,2522094,0,415597,0,0,55.42846,36.86983,24935836,8669,3,1,1,1496150163
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,2805045,0,415597,0,0,55.42846,36.86983,24935839,3746,3,1,1,1496150378


In [9]:
joined_data["Count"] = 1
stat = joined_data.groupby(["NAME"], as_index=False).agg({"Count": sum})
stat.head()

,NAME,Count
0,"""Мосрентген""",5902
1,Академический,25388
2,Алексеевский,24410
3,Алтуфьевский,7529
4,Арбат,25139


In [10]:
result = gpd.GeoDataFrame(pd.merge(stat, shapes, on="NAME"))
result.head()

,NAME,Count,geometry
0,"""Мосрентген""",5902,"POLYGON ((37.43956 55.62731, 37.44018 55.63042..."
1,Академический,25388,"POLYGON ((37.55226 55.68403, 37.55232 55.68408..."
2,Алексеевский,24410,"POLYGON ((37.63499 55.79714, 37.63552 55.79965..."
3,Алтуфьевский,7529,"POLYGON ((37.5674 55.8873, 37.57245 55.88724, ..."
4,Арбат,25139,"POLYGON ((37.57206 55.75202, 37.57396 55.75256..."


In [11]:
import fiona; fiona.supported_drivers

{'ARCGEN': 'r',
 'AeronavFAA': 'r',
 'BNA': 'raw',
 'DGN': 'raw',
 'DXF': 'raw',
 'ESRI Shapefile': 'raw',
 'GPKG': 'rw',
 'GPSTrackMaker': 'raw',
 'GPX': 'raw',
 'GeoJSON': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'OpenFileGDB': 'r',
 'PCIDSK': 'r',
 'SEGY': 'r',
 'SUA': 'r'}

In [12]:
result.to_file("result_5m.geojson", encoding="utf-8", driver="GeoJSON")

In [27]:
!open ./result.geojson -a tableau

![opt](options.png)
![viz](viz.png)